In [ ]:
import numpy as np
import pandas as pd
pd.options.mode.chained_assignment = None
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import StratifiedKFold

from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
import os

pd.set_option('display.max_columns', None)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
df_full = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Data Mining/Data/df_full.csv')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
df_full.head()

,_TOTINDA,_EXTETH3,_DENVST3,_LTASTH1,_CASTHM1,_DRDXAR2,_RACEG22,_AGE65YR,WTKG3,_INCOMG1,_RFMAM22,_RFSMOK3,DRNKANY6,_DRNKWK2,_AIDTST4,_ASTHMS1__1.0,_ASTHMS1__2.0,_ASTHMS1__3.0,_ASTHMS1__9.0,_EDUCAG__1.0,_EDUCAG__2.0,_EDUCAG__3.0,_EDUCAG__4.0,_EDUCAG__9.0,_SMOKER3__1.0,_SMOKER3__2.0,_SMOKER3__3.0,_SMOKER3__4.0,_SMOKER3__9.0,DEAF,BLIND,DECIDE,DIFFWALK,DIFFDRES,DIFFALON,LCSCTSC1,HIVTST7,SEXVAR,HHADULT,PHYSHLTH,MENTHLTH,POORHLTH,PRIMINSR,MEDCOST1,EXERANY2,LASTDEN4,RMVTETH4,CVDSTRK3,ASTHMA3,CHCCOPD3,ADDEPEV3,HAVARTH4,EDUCA,RENTHOM1,EMPLOY1,INCOME3,EMTSUPRT,SDHISOLT,SDHEMPLY,FOODSTMP,SDHFOOD1,SDHBILLS,SDHUTILS,Dissatisfied,Satisfied,Very Dissatisfied,Very satisfied,SDHSTRE1,RaceTreat-Better,RaceTreat-Better or Worse,RaceTreat-Only Encounter Same,RaceTreat-Same,RaceTreat-Worse,RaceTreat2-Better or Worse,RaceTreat2-Better than other,RaceTreat2-Only Encounter Same,RaceTreat2-Same as other,RaceTreat2-Worse than other,Health-Fair or Poor,Health-Good or Better,Health not good:0day,Health not good:1-13day,Health not good:14+day,mental not good:0day,mental not good:1-13day,mental not good:14+day,Class
0,1.0,0.0,1.0,1.0,1.0,0.0,1.0,1.0,63.500000,7.0,0.0,1.0,1.0,2.300000e-01,0.0,False,False,True,False,False,False,False,True,False,False,False,False,True,False,0,0,0,0,0,0,1,0,1,2,7,5,8,3,0,1,1,1,0,0,0,0,0,6,1,7,11,1.0,5.0,0,0,5.0,0,0,0,0,0,1,5.0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,1,0,0,Mild CD
1,1.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0,77.110000,5.0,0.0,0.0,0.0,5.397605e-81,0.0,False,False,True,False,False,False,False,True,False,True,False,False,False,False,0,0,0,0,0,0,0,0,0,2,7,5,8,3,0,1,2,1,0,0,0,0,1,6,1,2,7,1.0,4.0,1,0,5.0,0,1,0,0,0,1,5.0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,1,0,0,No CD
2,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,83.835752,6.0,0.0,1.0,1.0,2.300000e-01,0.0,False,False,True,False,False,False,False,True,False,False,False,False,True,False,0,0,1,0,0,0,1,0,1,2,4,5,4,3,0,1,1,1,0,0,0,1,1,6,1,7,9,3.0,4.0,0,0,5.0,0,0,0,0,0,1,4.0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,Mild CD
3,1.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,102.060000,4.0,0.0,1.0,1.0,4.700000e-01,1.0,False,False,True,False,False,False,False,True,False,False,False,False,True,False,0,0,0,1,0,0,1,1,1,2,1,3,8,2,0,1,1,1,0,0,0,0,0,6,1,7,6,2.0,4.0,0,0,5.0,0,0,0,1,0,0,4.0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,1,0,No CD
4,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,77.110000,4.0,1.0,1.0,1.0,8.000000e+00,1.0,False,False,True,False,False,False,True,False,False,False,False,True,False,False,1,0,0,0,0,0,1,1,0,2,7,5,8,3,0,1,1,2,0,0,0,0,1,5,1,7,6,5.0,3.0,0,0,5.0,0,0,0,1,0,0,4.0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,1,0,0,No CD


In [ ]:
df_full.shape

(51099, 87)

In [ ]:
# Assuming df_full is your DataFrame

# List of columns to convert
columns_to_convert = ['_ASTHMS1__1.0', '_ASTHMS1__2.0', '_ASTHMS1__3.0', '_ASTHMS1__9.0',
                      '_EDUCAG__1.0', '_EDUCAG__2.0', '_EDUCAG__3.0', '_EDUCAG__4.0', '_EDUCAG__9.0',
                      '_SMOKER3__1.0', '_SMOKER3__2.0', '_SMOKER3__3.0', '_SMOKER3__4.0', '_SMOKER3__9.0']

# Convert True/False to 1/0
df_full[columns_to_convert] = df_full[columns_to_convert].astype(int)

# Display the first few rows of the DataFrame
df_full.head()


,_TOTINDA,_EXTETH3,_DENVST3,_LTASTH1,_CASTHM1,_DRDXAR2,_RACEG22,_AGE65YR,WTKG3,_INCOMG1,_RFMAM22,_RFSMOK3,DRNKANY6,_DRNKWK2,_AIDTST4,_ASTHMS1__1.0,_ASTHMS1__2.0,_ASTHMS1__3.0,_ASTHMS1__9.0,_EDUCAG__1.0,_EDUCAG__2.0,_EDUCAG__3.0,_EDUCAG__4.0,_EDUCAG__9.0,_SMOKER3__1.0,_SMOKER3__2.0,_SMOKER3__3.0,_SMOKER3__4.0,_SMOKER3__9.0,DEAF,BLIND,DECIDE,DIFFWALK,DIFFDRES,DIFFALON,LCSCTSC1,HIVTST7,SEXVAR,HHADULT,PHYSHLTH,MENTHLTH,POORHLTH,PRIMINSR,MEDCOST1,EXERANY2,LASTDEN4,RMVTETH4,CVDSTRK3,ASTHMA3,CHCCOPD3,ADDEPEV3,HAVARTH4,EDUCA,RENTHOM1,EMPLOY1,INCOME3,EMTSUPRT,SDHISOLT,SDHEMPLY,FOODSTMP,SDHFOOD1,SDHBILLS,SDHUTILS,Dissatisfied,Satisfied,Very Dissatisfied,Very satisfied,SDHSTRE1,RaceTreat-Better,RaceTreat-Better or Worse,RaceTreat-Only Encounter Same,RaceTreat-Same,RaceTreat-Worse,RaceTreat2-Better or Worse,RaceTreat2-Better than other,RaceTreat2-Only Encounter Same,RaceTreat2-Same as other,RaceTreat2-Worse than other,Health-Fair or Poor,Health-Good or Better,Health not good:0day,Health not good:1-13day,Health not good:14+day,mental not good:0day,mental not good:1-13day,mental not good:14+day,Class
0,1.0,0.0,1.0,1.0,1.0,0.0,1.0,1.0,63.500000,7.0,0.0,1.0,1.0,2.300000e-01,0.0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,1,0,1,2,7,5,8,3,0,1,1,1,0,0,0,0,0,6,1,7,11,1.0,5.0,0,0,5.0,0,0,0,0,0,1,5.0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,1,0,0,Mild CD
1,1.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0,77.110000,5.0,0.0,0.0,0.0,5.397605e-81,0.0,0,0,1,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,2,7,5,8,3,0,1,2,1,0,0,0,0,1,6,1,2,7,1.0,4.0,1,0,5.0,0,1,0,0,0,1,5.0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,1,0,0,No CD
2,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,83.835752,6.0,0.0,1.0,1.0,2.300000e-01,0.0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,1,0,0,0,1,0,1,2,4,5,4,3,0,1,1,1,0,0,0,1,1,6,1,7,9,3.0,4.0,0,0,5.0,0,0,0,0,0,1,4.0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,Mild CD
3,1.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,102.060000,4.0,0.0,1.0,1.0,4.700000e-01,1.0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,1,1,1,2,1,3,8,2,0,1,1,1,0,0,0,0,0,6,1,7,6,2.0,4.0,0,0,5.0,0,0,0,1,0,0,4.0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,1,0,No CD
4,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,77.110000,4.0,1.0,1.0,1.0,8.000000e+00,1.0,0,0,1,0,0,0,1,0,0,0,0,1,0,0,1,0,0,0,0,0,1,1,0,2,7,5,8,3,0,1,1,2,0,0,0,0,1,5,1,7,6,5.0,3.0,0,0,5.0,0,0,0,1,0,0,4.0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,1,0,0,No CD


In [ ]:
df_full['Class'].unique()

array(['Mild CD', 'No CD', 'Severe CD', 'Moderate CD'], dtype=object)

In [ ]:
# Assuming df_full is your DataFrame
class_mapping = {'No CD': 0, 'Mild CD': 1, 'Moderate CD': 2, 'Severe CD': 3}
df_full['Class'] = df_full['Class'].replace(class_mapping)

df_full['Class'].unique()


array([1, 0, 3, 2])

In [ ]:
correlation_matrix = df_full.corr()

# Extract the correlation of cognitive decline with other variables
cognitive_decline_corr = correlation_matrix['Class'].drop('Class')
print("Correlation of Cognitive Decline with other variables:")
print(cognitive_decline_corr)


Correlation of Cognitive Decline with other variables:
_TOTINDA                  -0.094666
_EXTETH3                  -0.028602
_DENVST3                  -0.038144
_LTASTH1                  -0.048056
_CASTHM1                  -0.022572
                             ...   
Health not good:1-13day    0.018911
Health not good:14+day     0.240251
mental not good:0day      -0.224600
mental not good:1-13day    0.052024
mental not good:14+day     0.264311
Name: Class, Length: 86, dtype: float64


In [ ]:
 df_full.groupby(['Class']).size()

Class
0    45645
1     3010
2     1712
3      732
dtype: int64

In [ ]:
# Get the top 10 columns with the highest number of missing values
top_missing = missing_values.nlargest(10)

# Display the top 10 columns with the highest number of missing values
print(top_missing)



SDHSTRE1    8953
_DRDXAR2     217
_TOTINDA       0
_EXTETH3       0
_DENVST3       0
_LTASTH1       0
_CASTHM1       0
_RACEG22       0
_AGE65YR       0
WTKG3          0
dtype: int64


In [ ]:
# Get the unique values in the 'SDHSTRE1' column
unique_values = df_full['SDHSTRE1'].unique()

# Display the unique values
print(unique_values)



[ 5.  4.  3. nan  1.  2.]


In [ ]:
# Calculate the mode of the 'SDHSTRE1' column
mode_value = df_full['SDHSTRE1'].mode()[0]

# Replace missing values with the mode
df_full['SDHSTRE1'] = df_full['SDHSTRE1'].fillna(mode_value)

# Display the unique values after replacing missing values
unique_values_after_replace = df_full['SDHSTRE1'].unique()
print(unique_values_after_replace)


[5. 4. 3. 1. 2.]


In [ ]:
# Get the unique values in the 'SDHSTRE1' column
unique_values = df_full['_DRDXAR2'].unique()

# Display the unique values
print(unique_values)



[ 0.  1. nan]


In [ ]:
# Calculate the mode of the 'SDHSTRE1' column
mode_value = df_full['_DRDXAR2'].mode()[0]

# Replace missing values with the mode
df_full['_DRDXAR2'] = df_full['_DRDXAR2'].fillna(mode_value)

# Display the unique values after replacing missing values
unique_values_after_replace = df_full['_DRDXAR2'].unique()
print(unique_values_after_replace)


[0. 1.]


In [ ]:

from google.colab import files
df_full.to_csv('df_full.csv', index=False)
files.download('df_full.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
X = df_full.drop(columns=["Class"])
y = df_full["Class"]

In [32]:
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Standardize the features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Apply Linear Discriminant Analysis (LDA)
lda = LinearDiscriminantAnalysis()
X_train_lda = lda.fit_transform(X_train_scaled, y_train)
X_test_lda = lda.transform(X_test_scaled)

# Now you can use X_train_lda and X_test_lda as your transformed features


In [34]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

# Apply Logistic Regression
logreg = LogisticRegression()
logreg.fit(X_train_lda, y_train)

# Predict on the test set
y_pred = logreg.predict(X_test_lda)

# Calculate accuracy
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

conf_matrix = confusion_matrix(y_test, y_pred)
print("Confusion Matrix:")
print(conf_matrix)

Accuracy: 0.8908023483365949
Confusion Matrix:
[[9018   15   44   27]
 [ 603    6    7    3]
 [ 282    8   36   20]
 [  83    2   22   44]]


In [35]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, confusion_matrix

# Apply Random Forest
random_forest = RandomForestClassifier()
random_forest.fit(X_train_lda, y_train)

# Predict on the test set
y_pred = random_forest.predict(X_test_lda)

# Calculate accuracy
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

# Print confusion matrix
conf_matrix = confusion_matrix(y_test, y_pred)
print("Confusion Matrix:")
print(conf_matrix)


Accuracy: 0.8874755381604696
Confusion Matrix:
[[8987   29   63   25]
 [ 586   12   17    4]
 [ 280    8   37   21]
 [  90    2   25   34]]
